In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from IPython.display import clear_output

import sys
sys.path.append('../..')

from vae import VAEModel, compute_fid_score
from rbm import RBM
from dbn import DBN
from load_data import load_mnist

# Load data

In [2]:
X_train, y_train, X_test, y_test = load_mnist(binarize=True)

# VAE - Image Generation

In [3]:
PATH = './vae_fid_scores.pkl'

def load_dict(path=PATH):
    with open(path, 'rb') as handle:
        d = pickle.load(handle)
    return d

def save_dict(d, path=PATH):
    with open(path, 'wb') as handle:
        pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
### Load and sort the dictionary of results ###

dict_results = load_dict()
sorted_values = sorted(dict_results.items(), key=lambda item: item[1])  # Sort by the second element (value)
dict_results = dict(sorted_values)

In [5]:
best_parameters = list(dict_results.keys())[0]
beta = float(best_parameters.split(',')[0])
latent_dim = int(best_parameters.split(',')[1])
alpha_lrelu = float(best_parameters.split(',')[2])
print(f'Beta: {beta}, latent dimension: {latent_dim}, leaky_relu scope: {alpha_lrelu}')

Beta: 0.01, latent dimension: 2, leaky_relu scope: 0.2


In [6]:
vae_model = VAEModel(latent_dim=latent_dim, beta=beta, alpha_lrelu=alpha_lrelu)

In [7]:
nb_params_vae = 0
for param in vae_model.parameters():
    nb_params_vae += np.prod(param.size())

print(f"Nb parameters for VAE: {nb_params_vae:,}")

Nb parameters for VAE: 1,068,820


In [8]:
_ = vae_model.train_vae(X_train, 1000, learning_rate=3e-4, batch_size=256, print_error_every=10, patience=30)

Epoch 0: error = 218.2041


In [ ]:
vae_gen = vae_model.generate_data()

In [ ]:
vae_fid_score = vae_model.compute_fid_score(X_test)
print(f"FID score with VAE: {vae_fid_score:.4f}")

# DBN

In [ ]:
def compute_params_rbm(rbm):
    return rbm.a.shape[0] + rbm.b.shape[0] + np.prod(rbm.W.shape)

In [ ]:
dbn_model = DBN(X=X_train, L=5, qs=[128, 512, 784, 512, 256], use_adam=True)

In [ ]:
nb_params_dbn = 0
for rbm in dbn_model.rbms:
    nb_params_dbn += compute_params_rbm(rbm)

print(f"nb parameters for DBN: {nb_params_dbn:,}")

nb parameters for DBN: 1,104,688


In [ ]:
dbn_model.train_DBN(epochs=1000, learning_rate=3e-4, batch_size=128)

In [ ]:
_ = dbn_model.generer_image_DBN(num_samples=20, gibbs_steps=1000, ncols=10, image_size=(28, 28))

In [ ]:
dbn_generated_images = dbn_model.generer_image_DBN(num_samples=len(X_test), gibbs_steps=1000, plot_=False)
dbn_fid_score = compute_fid_score(X_test, dbn_generated_images, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
print(f"FID score with DBN: {dbn_fid_score:.4f}")

# RBM

In [ ]:
rbm_model = RBM(X=X_train, q=784+512+128, use_adam=True)

nb_params_rbm = compute_params_rbm(rbm_model)
print(f"nb parameters for RBM: {nb_params_rbm:,}")

nb parameters for RBM: 1,118,624


In [ ]:
_ = rbm_model.train_RBM(1000, learning_rate=3e-4, batch_size=256, plot_errors=True)

In [ ]:
_ = rbm_model.generer_image_RBM(nb_step_gibbs=1000, nb_to_generate=20, ncols=10, image_size=(28, 28))

In [ ]:
rbm_generated_images = rbm_model.generer_image_RBM(nb_step_gibbs=1000, nb_to_generate=len(X_test), ncols=10, plot_=False)
rbm_fid_score = compute_fid_score(X_test, rbm_generated_images, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
print(f"FID score with RBM: {dbn_fid_score:.4f}")